In [3]:
!pip install scikit-surprise

Сделал исправления.
1. на полном датасете
2. Подбор кол соседей. Хотел пошире диапазон заложить, но долго считается.

In [0]:
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

import pandas as pd

In [0]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [6]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1.256677e+09
1,1,481,3.5,1.256677e+09
2,1,1091,1.5,1.256677e+09
3,1,1257,4.5,1.256677e+09
4,1,1449,4.5,1.256677e+09


In [0]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [8]:
movies_with_ratings[movies_with_ratings.userId == 2.0].title.unique()

array(['Hackers (1995)', 'Escape from L.A. (1996)',
       'Sex, Lies, and Videotape (1989)', 'Harold and Maude (1971)',
       'Manhattan (1979)', 'Room with a View, A (1986)', 'Stripes (1981)',
       'Driving Miss Daisy (1989)', 'L.A. Story (1991)',
       'Broadcast News (1987)', 'Big Chill, The (1983)',
       'Arlington Road (1999)', 'Little Shop of Horrors (1986)',
       'Risky Business (1983)', 'American Graffiti (1973)'], dtype=object)

In [0]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [10]:
dataset.head()

,uid,iid,rating
0,4.0,Toy Story (1995),4.0
1,10.0,Toy Story (1995),5.0
2,14.0,Toy Story (1995),4.5
3,15.0,Toy Story (1995),4.0
4,22.0,Toy Story (1995),4.0


In [11]:
ratings.rating.min()

0.5

In [12]:
ratings.rating.max()

5.0

In [0]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [0]:
trainset, testset = train_test_split(data, test_size=.15)

In [15]:
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [0]:
test_pred = algo.test(testset)

In [17]:
accuracy.rmse(test_pred, verbose=False)

0.8811086766597184

Подберем кол соседей

In [18]:
amin_=10.

for i in range(3,70):
    algo = KNNWithMeans(k=i, sim_options={'name': 'pearson_baseline', 'user_based': True},verbose=False)
    algo.fit(trainset) 
    test_pred = algo.test(testset)
    if amin_ > accuracy.rmse(test_pred, verbose=False):
        amin_=accuracy.rmse(test_pred, verbose=False)
        kmin_=i  

print("При кол соседей = ",kmin_," достигается мин rmse =",amin_)


При кол соседей =  42  достигается мин rmse = 0.8809732562162853


--------------------------------------------------------------------------

In [0]:
algo = KNNWithMeans(k=i, sim_options={'name': 'pearson_baseline', 'user_based': True},verbose=False)
algo.fit(trainset) 
test_pred = algo.test(testset)

In [20]:
algo.predict(uid=2, iid='Fight Club (1999)')

Prediction(uid=2, iid='Fight Club (1999)', r_ui=None, est=4.382325949672155, details={'actual_k': 56, 'was_impossible': False})

In [21]:
algo = KNNBasic()
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [22]:
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.9392


0.9391744162894375

In [23]:
amin_=10.

for i in range(3,70):
    algo = KNNBasic(k=i, sim_options={'name': 'pearson_baseline', 'user_based': True},verbose=False)
    algo.fit(trainset)
    test_pred = algo.test(testset)
    if amin_ > accuracy.rmse(test_pred, verbose=False):
        amin_=accuracy.rmse(test_pred, verbose=False)
        kmin_=i  

print("При кол соседей = ",kmin_," достигается мин rmse =",amin_)

При кол соседей =  37  достигается мин rmse = 0.9546932720885982


In [0]:
algo = KNNBasic(k=kmin_, sim_options={'name': 'pearson_baseline', 'user_based': True},verbose=False)
algo.fit(trainset)
test_pred = algo.test(testset)

In [0]:
uid = 2
iid = 'Fight Club (1999)'

In [26]:
test_pred = algo.predict(uid, iid, r_ui=4, verbose=True)

user: 2          item: Fight Club (1999) r_ui = 4.00   est = 4.14   {'actual_k': 37, 'was_impossible': False}


In [27]:
test_pred

Prediction(uid=2, iid='Fight Club (1999)', r_ui=4, est=4.140903063384571, details={'actual_k': 37, 'was_impossible': False})